<a href="https://colab.research.google.com/github/bharadwaj-vedula/Direct-Marketing-Optimisation/blob/main/Rebel_Foods_Modeling_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   First model to classify if people will respond to mutual fund or not.
2.   Second model to classify if which consumer will respond to credit card
3.   Third model to classify which consumer will respond to Consumer Loan


---


4. Fourth model to if responds to mutual fund what will be expected revenue. (Train on all the folks who have mutual funds as it is a regression task)

5. Similarly for Credit card and Consumer Loans

# Imports

In [1]:
!pip install Optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.0 MB/s eta 0:00:00


In [2]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import pandas as pd
import numpy as np
from collections import Counter

import optuna

import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold,RepeatedStratifiedKFold,cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report,f1_score,average_precision_score,roc_auc_score

# Paths

In [4]:
class config:
  path = '/content/drive/MyDrive/Rebel Foods Assignment/DataScientist_CaseStudy_Dataset  (1) (1).xlsx'

In [6]:
df_dem = pd.read_excel(config.path,sheet_name = 'Soc_Dem')
df_actBalance = pd.read_excel(config.path,sheet_name = 'Products_ActBalance')
df_inflowOutflow = pd.read_excel(config.path, sheet_name = 'Inflow_Outflow')
df_salesRevenue = pd.read_excel(config.path,sheet_name = 'Sales_Revenues')

# Credit Card Propensity Model

### Data Perp

1. NaNs can be filled with zero as most of them are realted to count and balances.



In [7]:
cc_clients = df_salesRevenue.loc[df_salesRevenue['Sale_CC'] == 1]['Client'].tolist()

In [10]:
df = df_dem.merge(df_actBalance, how= 'inner',on='Client').merge(df_inflowOutflow,how= 'inner',on='Client').merge(df_salesRevenue,how= 'inner',on='Client')


In [11]:
sexMapping = {'F':np.int16(0),'M':np.int16(1)}
df['Sex'] = df['Sex'].map(sexMapping)
df.fillna(0,inplace= True)

In [12]:
df.drop(columns = ['Sale_MF', 'Sale_CL','Revenue_MF', 'Revenue_CC', 'Revenue_CL'], inplace= True)

In [13]:
df.columns

Index(['Client', 'Sex', 'Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF',
       'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA',
       'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred',
       'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb',
       'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder', 'Sale_CC'],
      dtype='object')

In [14]:
print(df['Sale_CC'].value_counts(dropna= False))
print(df_salesRevenue['Sale_CC'].value_counts(dropna= False))

Sale_CC
0    714
1    237
Name: count, dtype: int64
Sale_CC
0    727
1    242
Name: count, dtype: int64


In [23]:
df

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_CC
0,1217,1.0,38,165,1,0.0,0.0,0.0,0.0,0.0,...,714.285714,0.000000,0.000000,714.285714,1,1,0,0,1,0
1,850,0.0,49,44,1,0.0,0.0,0.0,0.0,0.0,...,1226.345357,0.000000,0.000000,121.928571,6,6,0,0,1,1
2,1473,1.0,54,34,1,1.0,0.0,0.0,1.0,1.0,...,3794.580714,357.142857,444.597500,2076.785714,48,38,1,26,11,0
3,1038,1.0,29,106,1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0
4,225,1.0,14,187,1,0.0,0.0,1.0,0.0,0.0,...,130.521429,0.000000,111.235714,19.285714,5,5,0,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,621,0.0,37,152,1,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0
947,1276,1.0,28,145,1,0.0,0.0,0.0,0.0,2.0,...,155.107143,35.714286,0.000000,0.000000,6,4,1,0,0,0
948,384,1.0,23,160,1,0.0,0.0,0.0,0.0,0.0,...,88.443929,50.000000,37.372500,1.071429,8,8,2,5,1,1
949,977,1.0,46,59,1,0.0,0.0,0.0,0.0,0.0,...,76.103571,75.000000,1.067857,0.035714,4,4,2,1,1,0


## Train Test Split

In [24]:
df_train , df_test = train_test_split( df, test_size= 0.2 , stratify = df['Sale_CC'])

In [25]:
df_train['Sale_CC'].value_counts()

,count
Sale_CC,
0,571
1,189


In [26]:
df_test['Sale_CC'].value_counts()

,count
Sale_CC,
0,143
1,48


In [27]:
X = df_train.iloc[:,1:-1]
y = df_train.iloc[:,-1]

In [28]:
X

,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
532,0.0,19,150,1,0.0,5.0,0.0,1.0,0.0,8.540357,...,8534.410714,5232.782143,1785.714286,699.589286,3045.464286,57,42,10,11,23
821,1.0,38,181,1,0.0,0.0,0.0,0.0,0.0,825.451071,...,571.421429,571.421429,207.142857,190.957143,171.428571,25,25,5,15,2
442,0.0,33,116,1,2.0,14.0,0.0,0.0,0.0,281.493571,...,402.750000,402.750000,0.000000,0.000000,399.071429,5,5,0,0,4
873,1.0,36,152,1,0.0,0.0,0.0,0.0,0.0,4143.043571,...,728.982143,728.982143,321.428571,26.232143,368.535714,9,9,2,3,2
142,1.0,37,7,1,0.0,0.0,1.0,1.0,0.0,2435.475714,...,1721.364286,1559.132143,392.857143,159.875000,1162.667857,28,18,3,6,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.0,68,42,1,0.0,0.0,0.0,0.0,0.0,1693.176429,...,934.358214,934.358214,285.714286,643.107143,0.000000,9,9,2,4,0
44,1.0,39,0,1,0.0,0.0,0.0,0.0,0.0,0.000000,...,768.625000,768.625000,107.142857,302.446429,357.785714,23,23,1,16,5
846,0.0,54,150,1,0.0,0.0,1.0,0.0,0.0,856.344643,...,1576.862143,1170.893571,357.142857,0.000000,406.000000,9,7,1,0,4
191,0.0,27,169,1,0.0,0.0,0.0,0.0,0.0,111.173214,...,699.967857,699.967857,209.820000,327.116786,151.785714,50,50,10,29,1


In [29]:
y.value_counts()

,count
Sale_CC,
0,571
1,189


In [30]:
X_test = df_test.iloc[:,1:-1]
y_test = df_test.iloc[:,-1]

X_test

,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
412,1.0,29,34,1,1.0,18.0,0.0,0.0,0.0,0.000000,...,1508.785714,80.214286,0.000000,80.214286,0.000000,2,1,0,1,0
910,0.0,38,71,1,0.0,0.0,0.0,0.0,0.0,0.000000,...,2113.277857,2113.277857,0.000000,463.069643,1649.029643,24,24,0,4,19
738,1.0,42,169,1,0.0,0.0,0.0,0.0,0.0,1.480714,...,1142.628571,1142.628571,0.000000,164.092857,975.142857,9,9,0,3,5
65,1.0,43,152,1,1.0,0.0,0.0,0.0,0.0,110.768571,...,386.442857,386.442857,307.142857,38.407143,35.714286,6,6,2,2,1
915,0.0,7,152,1,1.0,0.0,0.0,1.0,0.0,2164.632857,...,2560.158571,1928.307500,0.000000,698.518571,1271.068571,36,15,0,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,1.0,43,124,1,0.0,0.0,0.0,0.0,0.0,50.341071,...,362.036429,362.036429,217.857143,50.286429,92.464286,19,19,7,7,4
201,1.0,59,180,1,0.0,1.0,1.0,0.0,0.0,33.432143,...,664.535714,664.535714,178.571429,0.000000,485.107143,10,10,1,0,8
793,0.0,46,19,2,0.0,0.0,0.0,0.0,0.0,4.145357,...,111.714286,111.714286,0.000000,0.000000,108.678571,3,3,0,0,2
805,1.0,48,39,1,0.0,0.0,0.0,0.0,0.0,151.286786,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0


# XGBoost

In [ ]:
scores = {'f1_scores' : [], 'recall':[],'precision' : [],'accuracy':[],'auc_pr':[]}
for k,(train_idx, valid_idx) in enumerate(skf.split(X,y)):
  print(f'------------FOLD{k+1}------------')

  x_train = X.iloc[train_idx,1:].values
  y_train = y.iloc[train_idx].values

  x_valid = X.iloc[valid_idx,1:].values
  y_valid = y.iloc[valid_idx].values

  counter = Counter(y_train)
  weight = counter[0]/counter[1]

  # print(weight)

  clf = XGBClassifier()
  clf.fit(x_train,y_train)
  y_pred = clf.predict(x_valid)


  print(f'Accuracy score {accuracy_score(y_valid,y_pred)}')
  print(f'Recall score {recall_score(y_valid,y_pred)}')
  print(f'Precision score { precision_score(y_valid,y_pred)}')
  print(f'F1 score {f1_score(y_valid,y_pred)}')
  print(f'AUC PR {average_precision_score(y_valid,y_pred)}')
  print(f'Classification Report \n {classification_report(y_valid,y_pred)}')

  scores['f1_scores'].append(f1_score(y_valid,y_pred))
  scores['recall'].append(recall_score(y_valid,y_pred))
  scores['precision'].append(precision_score(y_valid,y_pred))
  scores['accuracy'].append(accuracy_score(y_valid,y_pred))
  scores['auc_pr'].append(average_precision_score(y_valid,y_pred))




In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
print(f'Mean F1 Scores {np.mean(scores["f1_scores"])}')
print(f'Mean Precision {np.mean(scores["precision"])}')
print(f'Mean Recall {np.mean(scores["recall"])}')
print(f'Mean AUC PR {np.mean(scores["auc_pr"])}')

Mean F1 Scores 0.20069449155335078
Mean Precision 0.3766666666666667
Mean Recall 0.14258064516129032
Mean AUC PR 0.23118958687040184


# Hyper parameter tuning

In [31]:
def objective(trial):
    """
    Objective function for Optuna to optimize.
    It trains an XGBoost Classifier with suggested hyperparameters and returns the
    cross-validated ROC AUC score.
    """
    param = {
        # 'objective': 'binary:logistic',
        'eval_metric': 'aucpr',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'tree_method': 'hist',
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-2, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 0.5, log=True),
        'lambda': trial.suggest_float('lambda', 1e-8, 100.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 100.0, log=True),
        'use_label_encoder': False,
        'random_state': 2025,
        'n_jobs': -1, #
        'early_stopping_rounds' : 50,


        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.6, 1.0)
    }


    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 0.0, 0.5)
        param['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 0.5)

    param['scale_pos_weight'] = trial.suggest_float(
        'scale_pos_weight',
        3.5 * 0.5,
        3.5 * 1.5,
        log=True
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)
    roc_auc_scores = []

    for train_index, val_index in skf.split(X, y):
        X_train_fold, X_val_fold = X.iloc[train_index,1:].values, X.iloc[val_index,1:].values
        y_train_fold, y_val_fold = y.iloc[train_index].values, y.iloc[val_index].values

        model = XGBClassifier(**param)

        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  verbose=False) # Set to True for verbose output during fitting

        val_preds_proba = model.predict(X_val_fold)

        if len(np.unique(y_val_fold)) == 2:
            f1 = f1_score(y_val_fold, val_preds_proba)
            roc_auc_scores.append(f1)
        else:

            continue

    if len(roc_auc_scores) > 0:
        return np.mean(roc_auc_scores)
    else:
        return 0.0

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=2025))
print("Starting Optuna hyperparameter tuning...")
study.optimize(objective, n_trials=50, show_progress_bar=True)
print("Optuna tuning complete.")


In [33]:
print("\nBest trial:")
trial = study.best_trial

print(f"  Value (Mean CV F1 Score): {trial.value:.4f}")
print("  Best Parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")



Best trial:
  Value (Mean CV F1 Score): 0.4274
  Best Parameters:
    booster: gbtree
    n_estimators: 93
    learning_rate: 0.010485498640488944
    max_depth: 8
    subsample: 0.6603849910975328
    colsample_bytree: 0.8337735587839497
    min_child_weight: 0.2826484185222205
    gamma: 0.00913231654354469
    lambda: 4.500846048393589
    alpha: 0.0001140661246759094
    colsample_bylevel: 0.8153362475890896
    colsample_bynode: 0.7903195379923754
    scale_pos_weight: 3.1651102279541097


In [34]:
best_params = trial.params
final_model = XGBClassifier(**best_params, use_label_encoder=False,
                                 eval_metric='aucpr',
                                 random_state=2025,
                                  n_jobs=-1,
                                  early_stopping_rounds = 50

                            )

print("\nTraining final model with best parameters...")
final_model.fit(X, y,
                eval_set=[(X_test, y_test)],
                verbose=True)



Training final model with best parameters...
[0]	validation_0-aucpr:0.24882
[1]	validation_0-aucpr:0.23386
[2]	validation_0-aucpr:0.29493
[3]	validation_0-aucpr:0.31158
[4]	validation_0-aucpr:0.32191
[5]	validation_0-aucpr:0.34408
[6]	validation_0-aucpr:0.30601
[7]	validation_0-aucpr:0.33221
[8]	validation_0-aucpr:0.33597
[9]	validation_0-aucpr:0.33132
[10]	validation_0-aucpr:0.33112
[11]	validation_0-aucpr:0.33621
[12]	validation_0-aucpr:0.31767
[13]	validation_0-aucpr:0.33214
[14]	validation_0-aucpr:0.36076
[15]	validation_0-aucpr:0.36667
[16]	validation_0-aucpr:0.36993
[17]	validation_0-aucpr:0.36729
[18]	validation_0-aucpr:0.36108
[19]	validation_0-aucpr:0.36215
[20]	validation_0-aucpr:0.36788
[21]	validation_0-aucpr:0.36253
[22]	validation_0-aucpr:0.37019
[23]	validation_0-aucpr:0.37065
[24]	validation_0-aucpr:0.37278
[25]	validation_0-aucpr:0.37150
[26]	validation_0-aucpr:0.36567
[27]	validation_0-aucpr:0.35696
[28]	validation_0-aucpr:0.35044
[29]	validation_0-aucpr:0.36910
[30]

XGBClassifier(alpha=0.0001140661246759094, base_score=None, booster='gbtree',
              callbacks=None, colsample_bylevel=0.8153362475890896,
              colsample_bynode=0.7903195379923754,
              colsample_bytree=0.8337735587839497, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric='aucpr', feature_types=None,
              gamma=0.00913231654354469, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=4.500846048393589,
              learning_rate=0.010485498640488944, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=0.2826484185222205, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=93,
              n_jobs=-1, ...)

In [35]:
print("\nEvaluating final model on test set:")

y_pred_proba = final_model.predict(X_test)
y_pred_class = (y_pred_proba > 0.5).astype(int) # Default threshold of 0.5

# ROC AUC
test_roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test ROC AUC: {test_roc_auc:.4f}")

# Precision, Recall, F1-Score
test_precision = precision_score(y_test, y_pred_class)
test_recall = recall_score(y_test, y_pred_class)
test_f1 = f1_score(y_test, y_pred_class)

print(f"Test Precision (at 0.5 threshold): {test_precision:.4f}")
print(f"Test Recall (at 0.5 threshold): {test_recall:.4f}")
print(f"Test F1-Score (at 0.5 threshold): {test_f1:.4f}")


Evaluating final model on test set:
Test ROC AUC: 0.5304
Test Precision (at 0.5 threshold): 0.2941
Test Recall (at 0.5 threshold): 0.3125
Test F1-Score (at 0.5 threshold): 0.3030


# Saving Model

In [69]:
final_model.save_model('/content/drive/MyDrive/Rebel Foods Assignment/Models/XGBClassifier_CC.json')